# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
#Импортирование библиотек
import requests
import pandas as pd
import sqlite3
import time

In [10]:
#Создание подлкючения к базе данных
conn = sqlite3.connect('our_home_data')
all_result = []


## Список объектов

In [3]:
def api_get(offset_, limit_):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_data.keys()
    objects_data.get('data').keys()
    objects_data.get('data').get('list')
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    return objids

In [4]:
all_result.extend(api_get (0, 5000))
all_result.extend(api_get (5001, 5000))
all_result.extend(api_get (10001, 5000))
len(all_result)

10526

# Данные по определенному объекту

In [5]:
data = []

for i in range(len(all_result)):
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{all_result[i]}'
        res = requests.get(url)
        data.append (res.json())
    except:
        print ('ошибка получения данных с сервера')
        pass

ошибка получения данных с сервера
ошибка получения данных с сервера
ошибка получения данных с сервера
ошибка получения данных с сервера
ошибка получения данных с сервера


In [6]:
data_norm = pd.json_normalize(data, max_level = 15)

In [7]:
data_norm.head(5)

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
0,0,29679,124647,7424,212-2,212-2,212-2,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29678,124648,7424,212-2,212-2,212-2,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data_norm = data_norm.drop(columns=['data.photoRenderDTO'])
data_norm = data_norm.drop(columns=['data.objectTransportInfo'])

# Cохранение в различных форматах

In [ ]:
data_norm = data_norm.applymap(str)

In [11]:
data_norm.to_sql('building', conn, if_exists = 'replace', index = False)
conn.commit()
conn.close

<function Connection.close>

In [12]:
data_norm.to_pickle('our_home_data.pkl')

In [13]:
data_norm.to_excel('our_home_data.xlsx')

# Получение изображений

In [14]:
def download_png(id_):
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(id_)
        res = requests.get(url)
        objects_data = res.json()
        objects_data.keys()
        url = objects_data['data']['photoRenderDTO'][0]['objRenderPhotoUrl']
        res = requests.get(url)
        out = open(str(id_)+".jpg", "wb")
        out.write(res.content)
        out.close()
    except:
        pass

In [19]:
#В данный момент стоит ограничение на скачивание картинок
for i in range (0,20):
    download_png(all_result[i])

[]